In [38]:
import re
import requests
from urllib.parse import urlsplit
import os 

def get_istio_auth_session(url: str, username: str, password: str) -> dict:
    """
    Determine if the specified URL is secured by Dex and try to obtain a session cookie.
    WARNING: only Dex `staticPasswords` and `LDAP` authentication are currently supported
             (we default default to using `staticPasswords` if both are enabled)

    :param url: Kubeflow server URL, including protocol
    :param username: Dex `staticPasswords` or `LDAP` username
    :param password: Dex `staticPasswords` or `LDAP` password
    :return: auth session information
    """
    # define the default return object
    auth_session = {
        "endpoint_url": url,    # KF endpoint URL
        "redirect_url": None,   # KF redirect URL, if applicable
        "dex_login_url": None,  # Dex login URL (for POST of credentials)
        "is_secured": None,     # True if KF endpoint is secured
        "session_cookie": None  # Resulting session cookies in the form "key1=value1; key2=value2"
    }

    # use a persistent session (for cookies)
    with requests.Session() as s:

        ################
        # Determine if Endpoint is Secured
        ################
        resp = s.get(url, allow_redirects=True,verify=os.getenv('cert_for_kubeflow'))
        if resp.status_code != 200:
            raise RuntimeError(
                f"HTTP status code '{resp.status_code}' for GET against: {url}"
            )

        auth_session["redirect_url"] = resp.url

        # if we were NOT redirected, then the endpoint is UNSECURED
        if len(resp.history) == 0:
            auth_session["is_secured"] = False
            return auth_session
        else:
            auth_session["is_secured"] = True

        ################
        # Get Dex Login URL
        ################
        redirect_url_obj = urlsplit(auth_session["redirect_url"])

        # if we are at `/auth?=xxxx` path, we need to select an auth type
        if re.search(r"/auth$", redirect_url_obj.path): 
            
            #######
            # TIP: choose the default auth type by including ONE of the following
            #######
            
            # OPTION 1: set "staticPasswords" as default auth type
            redirect_url_obj = redirect_url_obj._replace(
                path=re.sub(r"/auth$", "/auth/local", redirect_url_obj.path)
            )
            # OPTION 2: set "ldap" as default auth type 
            # redirect_url_obj = redirect_url_obj._replace(
            #     path=re.sub(r"/auth$", "/auth/ldap", redirect_url_obj.path)
            # )
            
        # if we are at `/auth/xxxx/login` path, then no further action is needed (we can use it for login POST)
        if re.search(r"/auth/.*/login$", redirect_url_obj.path):
            auth_session["dex_login_url"] = redirect_url_obj.geturl()

        # else, we need to be redirected to the actual login page
        else:
            # this GET should redirect us to the `/auth/xxxx/login` path
            resp = s.get(redirect_url_obj.geturl(), allow_redirects=True)
            if resp.status_code != 200:
                raise RuntimeError(
                    f"HTTP status code '{resp.status_code}' for GET against: {redirect_url_obj.geturl()}"
                )

            # set the login url
            auth_session["dex_login_url"] = resp.url

        ################
        # Attempt Dex Login
        ################
        resp = s.post(
            auth_session["dex_login_url"],
            data={"login": username, "password": password},
            allow_redirects=True
        )
        if len(resp.history) == 0:
            raise RuntimeError(
                f"Login credentials were probably invalid - "
                f"No redirect after POST to: {auth_session['dex_login_url']}"
            )

        # store the session cookies in a "key1=value1; key2=value2" string
        auth_session["session_cookie"] = "; ".join([f"{c.name}={c.value}" for c in s.cookies])
    return auth_session

/Users/yangwoolee/.pyenv/versions/3.9.1/lib/python3.9/site-packages/urllib3/connection.py:458: SubjectAltNameWarning: Certificate for localhost has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/urllib3/urllib3/issues/497 for details.)
  warnings.warn(


{'endpoint_url': 'https://localhost:8080',
 'redirect_url': 'https://localhost:8080/dex/auth/local/login?back=&state=wl7s347fz6k2wvzdxq3srplt3',
 'dex_login_url': 'https://localhost:8080/dex/auth/local/login?back=&state=wl7s347fz6k2wvzdxq3srplt3',
 'is_secured': True,
 'session_cookie': 'authservice_session=MTY3NDM3NzA5NXxOd3dBTkVnME0xZzFVMHRIU0ZSSVFVNVVOVU5hUkZFelREUk1Wa1JHTTFNeVRqTlNRek5LV1RSTVJ6VkJVakpHVkVkRVZreEZSMEU9fMOfQ430xor-x4Z2x8et14NygMDLekUvawt7kkMxQe_k'}

In [39]:
import requests
import kfp

HOST = "https://localhost:8080/"
USERNAME = "user@example.com"
PASSWORD = "12341234"

session = requests.Session()
response = session.get(HOST, verify=os.getenv('cert_for_kubeflow'))


headers = {
    "Content-Type": "application/x-www-form-urlencoded",
}

data = {"login": USERNAME, "password": PASSWORD}
session.post(response.url, headers=headers, data=data)
# session_cookie = session.cookies.get_dict()["authservice_session"]
session_cookie = session.cookies.get_dict()
print(session_cookie)

/Users/yangwoolee/.pyenv/versions/3.9.1/lib/python3.9/site-packages/urllib3/connection.py:458: SubjectAltNameWarning: Certificate for localhost has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/urllib3/urllib3/issues/497 for details.)
  warnings.warn(


{'authservice_session': 'MTY3NDM3NzA5OHxOd3dBTkUxUVdsUXpUVk15UVZoUlVGUTJVVFJKUmtkR1FsZE5SMHRUUjB0TlZFRktVRGRFTjBORFJqZEtORVV5UWxOT1IxVlBVa0U9fCr8G4-FLaEjZY6lSNnO7IPZDLvctiRMrhXdaF6HHSgy'}


In [37]:
import kfp
import os 

KUBEFLOW_ENDPOINT = "https://localhost:8080"
KUBEFLOW_USERNAME = "user@example.com"
KUBEFLOW_PASSWORD = "12341234"

auth_session = get_istio_auth_session(
    url=KUBEFLOW_ENDPOINT,
    username=KUBEFLOW_USERNAME,
    password=KUBEFLOW_PASSWORD
)

client_2 = kfp.Client(host=f"{KUBEFLOW_ENDPOINT}/pipeline", cookies=session_cookie['authservice_session'] ,ssl_ca_cert=os.getenv('cert_for_kubeflow'))
print('접속 : ',client_2.get_kfp_healthz())
client_1 = kfp.Client(host=f"{KUBEFLOW_ENDPOINT}/pipeline", cookies=auth_session["session_cookie"],ssl_ca_cert=os.getenv('cert_for_kubeflow'))
print('접속 : ',client_1.get_kfp_healthz())

/Users/yangwoolee/.pyenv/versions/3.9.1/lib/python3.9/site-packages/urllib3/connection.py:458: SubjectAltNameWarning: Certificate for localhost has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/urllib3/urllib3/issues/497 for details.)
  warnings.warn(


접속 :  {'multi_user': None}
접속 :  {'multi_user': True}


/Users/yangwoolee/.pyenv/versions/3.9.1/lib/python3.9/site-packages/urllib3/connection.py:458: SubjectAltNameWarning: Certificate for localhost has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/urllib3/urllib3/issues/497 for details.)
  warnings.warn(
/Users/yangwoolee/.pyenv/versions/3.9.1/lib/python3.9/site-packages/urllib3/connection.py:458: SubjectAltNameWarning: Certificate for localhost has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/urllib3/urllib3/issues/497 for details.)
  warnings.warn(
